# Carregar dados do Lakehouse Bronze para Silver

In [ ]:
# Definir o caminho ou referência da tabela Bronze (tabela "produto" no Lakehouse Bronze)

bronze_table = "Bronze_Lakehouse_GL.produto"  # Nome da tabela no Lakehouse Bronze

In [ ]:
# Carregar os dados da camada Bronze
df_bronze = spark.read.format("delta").table(bronze_table)  # Usando .table para ler diretamente de Delta Table

In [ ]:
# Exibir o schema para garantir que os dados foram carregados corretamente
df_bronze.printSchema()

# Transformações no DataFrame (opcional)

O objetivo da transformação para a camada Silver é:

**Limpeza de Dados:** Eliminar dados inválidos, nulos ou duplicados.

**Estruturação:** Organizar os dados de uma forma mais estruturada, como a separação de campos compostos ou a conversão de tipos de dados.

**Enriquecimento Básico:** Você pode adicionar dados adicionais, como a conversão de códigos em valores legíveis ou a adição de metadados para melhor compreensão dos dados.

**Normalização:** Ajustar os dados para garantir que tenham formatos e unidades consistentes

In [ ]:
# Realizar transformações, se necessário
## Exemplo:  df_transformed = df_bronze.select("column1", "column2")  # Exemplo de transformação

from pyspark.sql.functions import col

# Conversão de tipos de dados
df_transformed = df_bronze.withColumn("CUSTO_MEDIO", col("CUSTO_MEDIO").cast("double")) \
                          .withColumn("VALOR_UNITARIO", col("VALOR_UNITARIO").cast("double"))


In [ ]:
from pyspark.sql.functions import col, upper, expr

# Tratar valores nulos
df_transformed = df_transformed.fillna({
    "IDSK": "Desconhecido",
    "IDPRODUTO": "Desconhecido",
    "NOME": "Sem Nome",
    "ID_CATEGORIA": "Desconhecida",
    "CUSTO_MEDIO": 0.0,
    "VALOR_UNITARIO": 0.0
})

In [ ]:
from pyspark.sql.functions import col, upper, expr

# Padronização de texto
df_transformed = df_transformed.withColumn("NOME", upper(col("NOME"))) \
                               .withColumn("ID_CATEGORIA", upper(col("ID_CATEGORIA")))

In [ ]:
from pyspark.sql.functions import col, upper, expr


# Criar coluna derivada: Margem Bruta
df_transformed = df_transformed.withColumn("MARGEM_BRUTA", expr("VALOR_UNITARIO - CUSTO_MEDIO"))

In [ ]:
from pyspark.sql.functions import col, upper, expr


# Remover duplicatas
df_transformed = df_transformed.dropDuplicates(["IDPRODUTO", "IDSK"])

In [ ]:
from pyspark.sql.functions import col, upper, expr

# Validar valores numéricos
df_transformed = df_transformed.filter((col("CUSTO_MEDIO") >= 0) & (col("VALOR_UNITARIO") >= 0))

In [ ]:
from pyspark.sql.functions import col, upper, expr

# Exibir esquema e algumas linhas para verificar
df_transformed.printSchema()
df_transformed.show(5)

# Salvar no Lakehouse Silver

In [ ]:
# Definir o caminho ou referência da tabela Silver
silver_table = "Silver_Lakehouse_GL.produto"  # Nome da tabela no Lakehouse Silver

In [ ]:
# Gravar os dados na camada Silver
df_transformed.write.format("delta").mode("overwrite").saveAsTable(silver_table)  # Salva a tabela no formato Delta